In [ ]:
import csv
import json
import sys
import urllib.request

CONTEST = "W398"
CONTEST_URL = "https://leetcode.com/contest/weekly-contest-398/"
CONTEST_API_URL = "https://leetcode.com/contest/api/ranking/weekly-contest-398/"

headers = {
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-language': 'en-US,en;q=0.9',
    'sec-ch-ua': '"Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-origin',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
}

In [ ]:
PAGE = 81

user = []
for pagination in range(1, PAGE+1):
    for k in range(10):
        try:
            req = urllib.request.Request(CONTEST_API_URL+"?pagination=" + str(pagination) +"&region=global", headers=headers)
            r = urllib.request.urlopen(req).read()
            response = json.loads(r)
        except:
            pass
        else:
            break
    else:
        print(pagination)
        sys.exit()
    
    print(pagination)

    for i in range(25):
        user.append(response["total_rank"][i]["user_slug"])
    
print("finished!!!")

In [ ]:
QUESTION = "Q4"
CSV = "Q4.cpp.csv"

with open(CONTEST+"/"+CSV, "r") as f:
    rows = list(csv.reader(f))

DQ = 0
with open("markdown.txt", "w") as md:
    print("| rank | user | "+QUESTION+"   |\n| ---- | ---- | ---- |", file=md)
    
    with open(CONTEST+"/"+CSV, "w") as f:
        for i in rows:
            try:
                print(str(user.index(i[1])+1) + "," + i[1] + "," + i[2], file=f)
                print("|", user.index(i[1])+1, "| ["+i[1]+"](https://leetcode.com/u/"+i[1]+") |", CONTEST_URL+"submissions/detail/"+i[2], "|", file=md)
            except:
                DQ += 1
                print("DQ," + i[1] + "," + i[2], file=f)
                print("| DQ | ["+i[1]+"](https://leetcode.com/u/"+i[1]+") |", CONTEST_URL+"submissions/detail/"+i[2], "|", file=md)

print("DQ:", DQ)